<a href="https://colab.research.google.com/github/tnc-br/ddf-isoscapes/blob/universal_kriging/kriging/universal_kriging_interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install earthengine-api geemap geopandas geobr rtree pyproj rasterio contextily descartes -q

In [ ]:
!pip install pykrige -q

In [ ]:
## Import packages
import rasterio as rio
import rasterio.mask
from rasterio.plot import show
from rasterio.transform import Affine
import contextily as cx
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import geemap
import json
import seaborn as sns
from shapely.geometry import Point
import pykrige.kriging_tools as kt
from pykrige.uk import UniversalKriging
import os

In [ ]:
import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")

import ddfimport

ddfimport.ddf_source_control_pane()
# ddfimport.ddf_import_common()

In [ ]:
import importlib
import raster
importlib.reload(raster)

<module 'raster' from '/content/gdrive/MyDrive/main/ddf_common/raster.py'>

In [ ]:
# Required to both import raster and read GDrive files
raster.RASTER_BASE = "/MyDrive/amazon_rainforest_files/amazon_rasters/" #@param
raster.SAMPLE_DATA_BASE = "/MyDrive/amazon_rainforest_files/amazon_sample_data/" #@param
raster.TEST_DATA_BASE = "/MyDrive/amazon_rainforest_files/amazon_test_data/" #@param
raster.ANIMATIONS_BASE = "/MyDrive/amazon_rainforest_files/amazon_animations/" #@param
raster.GDRIVE_BASE = "/content/gdrive" #@param

In [ ]:
DEBUG = True #@param {type:"boolean"}
PROJECT_BASE = os.path.join(raster.GDRIVE_BASE, "MyDrive/amazon_rainforest_files")
# From PROJECT_BASE root
DATASET_PATH = "amazon_sample_data/uc_davis_2023_08_25_train_random_grouped.csv" #@param
# From PROJECT_BASE root
BIOME_PATH = "christian_files/BR_UF_2021.shp" #@param
# From PROJECT_BASE root
OUTPUT_MEANS_ISOSCAPE_PATH = "amazon_rasters/uc_davis_d18O_cel_universal_exponential_random_grouped_means.tiff" #@param
OUTPUT_VARS_ISOSCAPE_PATH = "amazon_rasters/uc_davis_d18O_cel_universal_exponential_random_grouped_vars.tiff" #@param
ISOTOPE_COLUMN_NAME = "d18O_cel_mean" #@param
VARIOGRAM_MODEL = "exponential" #@param

In [ ]:
# Access data stored on Google Drive
if raster.GDRIVE_BASE:
    raster.mount_gdrive()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
if DEBUG:
    %pip install -Uqq ipdb
    import ipdb
    %pdb on

In [ ]:
# Load coordinates and values from dataset path.
df = pd.read_csv(os.path.join(PROJECT_BASE, DATASET_PATH)).dropna(subset=['lat', 'long', ISOTOPE_COLUMN_NAME])
long = df['long'].values
lat = df['lat'].values
isotope_values = df[ISOTOPE_COLUMN_NAME].values

In [ ]:
def get_kriging_means_variances(
    bounds: raster.Bounds,
    long: list,
    lat: list,
    isotope_values: list,
    variogram_model: str = "linear"
):
  '''
  Gets the means and variances of an Ordinary Kriging model fit to the provided
  long, lat and isotope_values predicted on the coordinates within the bounds
  provided using the type of variogram_model specified.
  '''

  ok = UniversalKriging(
      long,
      lat,
      isotope_values,
      variogram_model=variogram_model,
      verbose=False,
      enable_plotting=True,
      pseudo_inv=True # this leads to more numerical stability and redundant
                      # points are averaged. This prevents singular matrix
                      # errors which happens frequently with variance isoscape
                      # in this method.
  )

  min_long, max_long, min_lat, max_lat = bounds.to_matplotlib()

  isoscape_long_values =  np.linspace(min_long, max_long, bounds.raster_size_x)
  isoscape_lat_values = np.linspace(min_lat, max_lat, bounds.raster_size_y)
  # Predict the isotope values on the range of lattitude and longitude values
  # within the bounds.
  means, variances  = ok.execute("grid", isoscape_long_values,
                                 isoscape_lat_values)
  ok.print_statistics()

  # Execute takes N longitude values and M latittude values and returns a
  # (M, N) matrix. Transpose to get (N, M) matrix so x and y are still longitude
  # and latitude respectively.
  return means.T, variances.T

In [ ]:
def display_isoscape(means_filename, vars_filename, gdf_biome):
  '''
  Given a filename to a RasterIO file containing an isoscape raster,
  display the isoscape masked by the gdf_biome geometry.
  '''
  # Mean Isotope Isoscape Band Image
  means_raster = rasterio.open(means_filename)
  means_image, means_transform = rio.mask.mask(means_raster, gdf_biome.geometry.values, crop = True)

  fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 20))
  means_raster_images = show(means_image, ax = ax1, transform = means_transform, cmap = "RdYlGn")
  means_raster_image = means_raster_images.get_images()[0]
  fig.colorbar(means_raster_image, ax=ax1, shrink=0.3)

  ax1.set_title("Mean Isotope Isoscape Band")

  # Variance Isotope Isoscape Band Image
  vars_raster = rasterio.open(vars_filename)
  vars_image, vars_transform = rio.mask.mask(vars_raster, gdf_biome.geometry.values, crop = True)

  vars_raster_images = show(vars_image, ax = ax2, transform = vars_transform, cmap = "RdYlGn")
  vars_raster_image = vars_raster_images.get_images()[0]
  fig.colorbar(vars_raster_image, ax=ax2, shrink=0.3)

  ax2.set_title("Variance Isotope Isoscape Band")
  plt.show()

In [ ]:
# TODO: Use standard raster dimensions and resolution.
atmosphere_isoscape_geotiff = raster.load_raster(raster.get_raster_path("brasil_clim_raster.tiff"))
bounds =  raster.get_extent(atmosphere_isoscape_geotiff.gdal_dataset)

In [ ]:
means, variances = get_kriging_means_variances(bounds, long, lat, isotope_values, variogram_model=VARIOGRAM_MODEL)

In [ ]:
out_filename_mean = os.path.join(PROJECT_BASE, OUTPUT_MEANS_ISOSCAPE_PATH)
out_filename_var = os.path.join(PROJECT_BASE, OUTPUT_VARS_ISOSCAPE_PATH)

means_masked_array = np.ma.masked_array(
    np.expand_dims(np.array(means), axis=2), mask=np.isnan(means))
variances_masked_array = np.ma.masked_array(
    np.expand_dims(np.array(variances), axis=2), mask=np.isnan(variances))

raster.save_numpy_to_geotiff(bounds, means_masked_array, out_filename_mean)
raster.save_numpy_to_geotiff(bounds, variances_masked_array, out_filename_var)

In [ ]:
# Load geometry dataset that'll be used to mask isoscape in expected shape.
biome_path = os.path.join(PROJECT_BASE, BIOME_PATH)
gdf_biome = gpd.read_file(biome_path)

/usr/local/lib/python3.10/dist-packages/geopandas/array.py:93: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [ ]:
display_isoscape(out_filename_mean, out_filename_var, gdf_biome)